In [1]:
% pylab inline
import tensorflow as tf
import numpy as np
# import matplotlib.pyplot as plt
from tqdm import tqdm
# from tensorflow.contrib.kfac.python.ops.utils import fwd_gradients
import seaborn as sns

slim = tf.contrib.slim
ds = tf.contrib.distributions

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.framework import ops
from tensorflow.python.training import optimizer

# Adapted from https://raw.githubusercontent.com/openai/iaf/master/tf_utils/adamax.py

class OptimisticMirrorDescentOptimizer(optimizer.Optimizer):


    def __init__(self, learning_rate=0.001, use_locking=False, name="OMD"):
        
        super(OptimisticMirrorDescentOptimizer, self).__init__(use_locking, name)
        self._lr = learning_rate
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None

    def _prepare(self):
        self._lr_t = ops.convert_to_tensor(self._lr, name="learning_rate")

    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        for v in var_list:
            self._zeros_slot(v, "g", self._name)
      
    def _apply_dense(self, grad, var):
        
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)

        g_t = grad
        g_t_1 = self.get_slot(var, "g")
        g_t = g_t_1.assign(g_t)

        var_update = state_ops.assign_sub(var, 2. * lr_t * g_t - lr_t * g_t_1) #Adam would be lr_t * g_t
        return control_flow_ops.group(*[var_update, g_t])
  
    def _apply_sparse(self, grad, var):
        raise NotImplementedError("Sparse gradient updates are not supported.")

In [3]:
print(tf.__version__)

1.3.0


In [4]:
def generator(z):
    with tf.variable_scope("generator"):
        theta = tf.get_variable(name='thetha',shape=z.shape)
    return z + theta

def discriminator(x, reuse=False):
    with tf.variable_scope("discriminator", reuse=reuse):
        w = tf.get_variable(name='thetha',shape=x.shape)
    return tf.reduce_sum(tf.multiply(x, w))

In [5]:
params = dict(
    batch_size=512,
    learning_rate=0.01,
    beta1=0.5,
    epsilon=1e-8,
    max_iter=800000,
    viz_every=1000,
    z_dim=2,
    x_dim=2,
    optimizer='omd', # rmsprop sgd sga
    reg_w=10.,
    v = (3.,4.)
)

In [6]:
tf.reset_default_graph()

data = ds.MultivariateNormalDiag(params['v'], tf.ones(params['z_dim'])).sample(params['batch_size'])
noise = ds.Normal(tf.zeros(params['z_dim']), tf.ones(params['z_dim'])).sample(params['batch_size'])

# Construct generator and discriminator nets
with slim.arg_scope([slim.fully_connected], weights_initializer=tf.orthogonal_initializer(gain=1.4)):
    samples = generator(noise)
    real_score = discriminator(data)
    fake_score = discriminator(samples, reuse=True)
    
# Saddle objective    
loss = tf.reduce_mean(real_score) - tf.reduce_mean(fake_score)

gen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "generator")
disc_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator")

with tf.name_scope('summary'):
        tf.summary.scalar('loss_discriminator', loss)
        tf.summary.scalar('loss_generator', -loss)
        summa = tf.summary.merge_all()

In [7]:
if params['optimizer'] == 'sgd':    
    d_train_opt = tf.train.GradientDescentOptimizer(params['learning_rate'])
    g_train_opt = tf.train.GradientDescentOptimizer(params['learning_rate'])
    d_train_op = d_train_opt.minimize(loss, var_list=disc_vars)
    g_train_op = g_train_opt.minimize(-loss, var_list=gen_vars)
    
if params['optimizer'] == 'omd':    
    d_train_opt = OptimisticMirrorDescentOptimizer(params['learning_rate'])
    g_train_opt = OptimisticMirrorDescentOptimizer(params['learning_rate'])
    d_train_op = d_train_opt.minimize(loss, var_list=disc_vars)
    g_train_op = g_train_opt.minimize(-loss, var_list=gen_vars)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
logdir= './logs/{}'.format(params['optimizer'])
writer = tf.summary.FileWriter(logdir, sess.graph)

In [9]:
for i in tqdm(range(params['max_iter']+1)):
    f, _, _ = sess.run([[loss], g_train_op, d_train_op])
    sm = sess.run(summa)
    writer.add_summary(sm, i)

 22%|███████████████▊                                                         | 172847/800001 [04:21<15:47, 662.01it/s]

KeyboardInterrupt: 

 22%|███████████████▊                                                         | 172847/800001 [04:40<16:56, 617.23it/s]